In [2]:
import glob
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from PIL import Image


import os
import io

from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import glob
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split

## SVM (128 x 128 - no resize)

### 1. Split testing, training

In [81]:
# print(os.getcwd())
current = os.getcwd()
data_folder = os.path.join(current, "normalized_images_no_resize_128")
chess_types_folders = glob.glob(os.path.join(data_folder, "*"))
# print(chess_types_folders)
pieces_info = []
labels = {"King": 1, "Knight":2, "Bishop":3, "Rook":4, "Pawn":5, "Queen":6}
for chess_types in chess_types_folders:
    pieces = glob.glob(f'{chess_types}/*')
    # print(pieces)
    type = chess_types.split("/")[-1]
    for piece in pieces:
        p = {"normalized_img": np.load(piece).reshape(-1), "label": labels[type]}
        pieces_info.append(p)
chess_df = pd.DataFrame(pieces_info)

In [82]:
chess_1 = chess_df[chess_df['label'] == 1]
chess_2 = chess_df[chess_df['label'] == 2]
chess_3 = chess_df[chess_df['label'] == 3]
chess_4 = chess_df[chess_df['label'] == 4]
chess_5 = chess_df[chess_df['label'] == 5]
chess_6 = chess_df[chess_df['label'] == 6]

In [83]:
train_df_1, test_df_1 = train_test_split(chess_1, test_size=0.3, random_state=1)
train_df_2, test_df_2 = train_test_split(chess_2, test_size=0.3, random_state=1)
train_df_3, test_df_3 = train_test_split(chess_3, test_size=0.3, random_state=1)
train_df_4, test_df_4 = train_test_split(chess_4, test_size=0.3, random_state=1)
train_df_5, test_df_5 = train_test_split(chess_5, test_size=0.3, random_state=1)
train_df_6, test_df_6 = train_test_split(chess_6, test_size=0.3, random_state=1)


In [84]:
train_dfs_list = [train_df_1, train_df_2, train_df_3, train_df_4, train_df_5, train_df_6]
test_dfs_list = [test_df_1, test_df_2, test_df_3, test_df_4, test_df_5, test_df_6]

train_df = pd.concat(train_dfs_list, ignore_index=True)
test_df = pd.concat(test_dfs_list, ignore_index=True)

In [85]:
# Load dataset
X_train, y_train = np.array(train_df['normalized_img']), np.array(train_df['label'])
X_test, y_test = np.array(test_df['normalized_img']), np.array(test_df['label'])

X_train = np.array([x for x in X_train])
X_test = np.array([x for x in X_test])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_test.shape

### 1. Linear SVM

In [ ]:
# Selecting best C for Linear SVM Model
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		lin_svm = SVC(kernel='linear',C=C)
		lin_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = lin_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = lin_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)

C = 0.01: (avg train error, avg validation error) = (0.0419, 0.0419)
C = 0.1: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 1.0: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 10: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [ ]:
# Fit linear model on best C
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='linear',C=C_chosen)
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"Linear SVM Performance Metrics with C={C_chosen}:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 0.1, with lowest avg test error: 0.0000
Final Test Error: 0.7692
Linear SVM Performance Metrics with C=0.1:
Accuracy: 0.2308
Precision: 0.2380
Recall: 0.2308
F1 Score: 0.2318

Confusion Matrix:
[[10  7  5  6  3  7]
 [ 6 13  6  4 13  5]
 [10  4  5  3 10 10]
 [12  3  8 12  9  2]
 [ 7  6  5  8 14  8]
 [ 9  5  8  4  7  6]]

Classification Report:
              precision    recall  f1-score   support

           1       0.19      0.26      0.22        38
           2       0.34      0.28      0.31        47
           3       0.14      0.12      0.13        42
           4       0.32      0.26      0.29        46
           5       0.25      0.29      0.27        48
           6       0.16      0.15      0.16        39

    accuracy                           0.23       260
   macro avg       0.23      0.23      0.23       260
weighted avg       0.24      0.23      0.23       260



### 2. RBF SVM

In [86]:
#Selecting best C for RBF SVM Model (gamma=auto)
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		auto_svm = SVC(kernel='rbf',C=C, gamma='auto')
		auto_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = auto_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = auto_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)

C = 0.01: (avg train error, avg validation error) = (0.7228, 0.7228)
C = 0.1: (avg train error, avg validation error) = (0.7228, 0.7228)
C = 1.0: (avg train error, avg validation error) = (0.3860, 0.3861)
C = 10: (avg train error, avg validation error) = (0.0492, 0.0492)
C = 100: (avg train error, avg validation error) = (0.0026, 0.0026)


In [89]:
#Fit RBF model on best C (gamma='auto')
lowest_val_error_index = np.argmin(cv_val_avg_score)
# C_chosen = C_values[lowest_val_error_index]
C_chosen = 1.0

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='rbf',C=C_chosen, gamma='auto')
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='auto':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))
# labels = {"King": 1, "Knight":2, "Bishop":3, "Rook":4, "Pawn":5, "Queen":6}


Best C = 1.0, with lowest avg test error: 0.0026
Final Test Error: 0.6706
RBF SVM Performance Metrics with C=1.0, gamma='auto':
Accuracy: 0.3294
Precision: 0.3592
Recall: 0.3294
F1 Score: 0.2973

Confusion Matrix:
[[12  1  7  0  2  1]
 [ 7 22  0  2  1  0]
 [ 9  5  5  1  7  0]
 [ 7  6  5  5  8  0]
 [ 9  6  3  4 11  0]
 [ 7  7  5  1  3  1]]

Classification Report:
              precision    recall  f1-score   support

           1       0.24      0.52      0.32        23
           2       0.47      0.69      0.56        32
           3       0.20      0.19      0.19        27
           4       0.38      0.16      0.23        31
           5       0.34      0.33      0.34        33
           6       0.50      0.04      0.08        24

    accuracy                           0.33       170
   macro avg       0.36      0.32      0.29       170
weighted avg       0.36      0.33      0.30       170



### 3. Custom SVM

## SVM (64 x 64)

### 1. Split testing, training

In [71]:
print(os.getcwd())
current = os.getcwd()
data_folder = os.path.join(current, "normalized_images_64")
chess_types_folders = glob.glob(os.path.join(data_folder, "*"))
# print(chess_types_folders)
pieces_info = []
labels = {"King": 1, "Knight":2, "Bishop":3, "Rook":4, "Pawn":5, "Queen":6}
for chess_types in chess_types_folders:
    pieces = glob.glob(f'{chess_types}/*')
    # print(pieces)
    type = chess_types.split("/")[-1]
    for piece in pieces:
        p = {"normalized_img": np.load(piece).reshape(-1), "label": labels[type]}
        pieces_info.append(p)
chess_df = pd.DataFrame(pieces_info)

/Users/khanhchile/Downloads/School/CSCI 5525/5525Chess


In [62]:
chess_1 = chess_df[chess_df['label'] == 1]
chess_2 = chess_df[chess_df['label'] == 2]
chess_3 = chess_df[chess_df['label'] == 3]
chess_4 = chess_df[chess_df['label'] == 4]
chess_5 = chess_df[chess_df['label'] == 5]
chess_6 = chess_df[chess_df['label'] == 6]

In [63]:
train_df_1, test_df_1 = train_test_split(chess_1, test_size=0.3, random_state=1)
train_df_2, test_df_2 = train_test_split(chess_2, test_size=0.3, random_state=1)
train_df_3, test_df_3 = train_test_split(chess_3, test_size=0.3, random_state=1)
train_df_4, test_df_4 = train_test_split(chess_4, test_size=0.3, random_state=1)
train_df_5, test_df_5 = train_test_split(chess_5, test_size=0.3, random_state=1)
train_df_6, test_df_6 = train_test_split(chess_6, test_size=0.3, random_state=1)


In [64]:
train_dfs_list = [train_df_1, train_df_2, train_df_3, train_df_4, train_df_5, train_df_6]
test_dfs_list = [test_df_1, test_df_2, test_df_3, test_df_4, test_df_5, test_df_6]

train_df = pd.concat(train_dfs_list, ignore_index=True)
test_df = pd.concat(test_dfs_list, ignore_index=True)

In [65]:
# Load dataset
X_train, y_train = np.array(train_df['normalized_img']), np.array(train_df['label'])
X_test, y_test = np.array(test_df['normalized_img']), np.array(test_df['label'])

X_train = np.array([x for x in X_train])
X_test = np.array([x for x in X_test])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### 1. Linear SVM

In [66]:
# Selecting best C for Linear SVM Model
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		lin_svm = SVC(kernel='linear',C=C)
		lin_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = lin_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = lin_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)

C = 0.01: (avg train error, avg validation error) = (0.0419, 0.0419)
C = 0.1: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 1.0: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 10: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [67]:
# Fit linear model on best C
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='linear',C=C_chosen)
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"Linear SVM Performance Metrics with C={C_chosen}:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 0.1, with lowest avg test error: 0.0000
Final Test Error: 0.7692
Linear SVM Performance Metrics with C=0.1:
Accuracy: 0.2308
Precision: 0.2380
Recall: 0.2308
F1 Score: 0.2318

Confusion Matrix:
[[10  7  5  6  3  7]
 [ 6 13  6  4 13  5]
 [10  4  5  3 10 10]
 [12  3  8 12  9  2]
 [ 7  6  5  8 14  8]
 [ 9  5  8  4  7  6]]

Classification Report:
              precision    recall  f1-score   support

           1       0.19      0.26      0.22        38
           2       0.34      0.28      0.31        47
           3       0.14      0.12      0.13        42
           4       0.32      0.26      0.29        46
           5       0.25      0.29      0.27        48
           6       0.16      0.15      0.16        39

    accuracy                           0.23       260
   macro avg       0.23      0.23      0.23       260
weighted avg       0.24      0.23      0.23       260



### 2. RBF SVM

In [68]:
#Selecting best C for RBF SVM Model (gamma=auto)
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		auto_svm = SVC(kernel='rbf',C=C, gamma='auto')
		auto_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = auto_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = auto_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)

C = 0.01: (avg train error, avg validation error) = (0.7232, 0.7232)
C = 0.1: (avg train error, avg validation error) = (0.7232, 0.7232)
C = 1.0: (avg train error, avg validation error) = (0.3054, 0.3053)
C = 10: (avg train error, avg validation error) = (0.0369, 0.0369)
C = 100: (avg train error, avg validation error) = (0.0017, 0.0017)


In [69]:
#Fit RBF model on best C (gamma='auto')
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='rbf',C=C_chosen, gamma='auto')
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='auto':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 100, with lowest avg test error: 0.0017
Final Test Error: 0.6769
RBF SVM Performance Metrics with C=100, gamma='auto':
Accuracy: 0.3231
Precision: 0.3239
Recall: 0.3231
F1 Score: 0.3208

Confusion Matrix:
[[10  8  8  0  9  3]
 [ 2 20  9  6  4  6]
 [ 7  5  7  5  9  9]
 [ 4  8  5 16 11  2]
 [ 5  5  5  6 21  6]
 [ 7  4  5  3 10 10]]

Classification Report:
              precision    recall  f1-score   support

           1       0.29      0.26      0.27        38
           2       0.40      0.43      0.41        47
           3       0.18      0.17      0.17        42
           4       0.44      0.35      0.39        46
           5       0.33      0.44      0.38        48
           6       0.28      0.26      0.27        39

    accuracy                           0.32       260
   macro avg       0.32      0.32      0.32       260
weighted avg       0.32      0.32      0.32       260



### 3. Custom SVM

## SVM (128 x 128)

### 1. Split testing, training

In [3]:
print(os.getcwd())
current = os.getcwd()
data_folder = os.path.join(current, "normalized_images_128")
chess_types_folders = glob.glob(os.path.join(data_folder, "*"))
# print(chess_types_folders)
pieces_info = []
labels = {"King": 1, "Knight":2, "Bishop":3, "Rook":4, "Pawn":5, "Queen":6}
for chess_types in chess_types_folders:
    pieces = glob.glob(f'{chess_types}/*')
    # print(pieces)
    type = chess_types.split("/")[-1]
    for piece in pieces:
        p = {"normalized_img": np.load(piece).reshape(-1), "label": labels[type]}
        pieces_info.append(p)
chess_df = pd.DataFrame(pieces_info)

/Users/khanhchile/Downloads/School/CSCI 5525/5525Chess


In [4]:
# chess_df.head()
chess_df['label'].value_counts()

label
5    157
2    156
4    152
3    137
6    128
1    126
Name: count, dtype: int64

In [33]:
chess_1 = chess_df[chess_df['label'] == 1]
chess_2 = chess_df[chess_df['label'] == 2]
chess_3 = chess_df[chess_df['label'] == 3]
chess_4 = chess_df[chess_df['label'] == 4]
chess_5 = chess_df[chess_df['label'] == 5]
chess_6 = chess_df[chess_df['label'] == 6]


In [34]:
train_df_1, test_df_1 = train_test_split(chess_1, test_size=0.3, random_state=1)
train_df_2, test_df_2 = train_test_split(chess_2, test_size=0.3, random_state=1)
train_df_3, test_df_3 = train_test_split(chess_3, test_size=0.3, random_state=1)
train_df_4, test_df_4 = train_test_split(chess_4, test_size=0.3, random_state=1)
train_df_5, test_df_5 = train_test_split(chess_5, test_size=0.3, random_state=1)
train_df_6, test_df_6 = train_test_split(chess_6, test_size=0.3, random_state=1)


In [35]:
train_dfs_list = [train_df_1, train_df_2, train_df_3, train_df_4, train_df_5, train_df_6]
test_dfs_list = [test_df_1, test_df_2, test_df_3, test_df_4, test_df_5, test_df_6]
# train_dfs_list = [train_df_1, train_df_3]
# test_dfs_list = [test_df_1, test_df_3]

In [36]:

train_df = pd.concat(train_dfs_list, ignore_index=True)
test_df = pd.concat(test_dfs_list, ignore_index=True)


In [37]:
# Load dataset
X_train, y_train = np.array(train_df['normalized_img']), np.array(train_df['label'])
X_test, y_test = np.array(test_df['normalized_img']), np.array(test_df['label'])

X_train = np.array([x for x in X_train])
X_test = np.array([x for x in X_test])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
X_test.shape

(260, 16384)

### 2. Linear SVM

In [10]:
# Selecting best C for Linear SVM Model
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		lin_svm = SVC(kernel='linear',C=C)
		lin_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = lin_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = lin_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)


C = 0.01: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 0.1: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 1.0: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 10: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [11]:
# Fit linear model on best C
lowest_val_error_index = np.argmin(cv_val_avg_score)
# C_chosen = C_values[lowest_val_error_index]
C_chosen = 100

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='linear',C=C_chosen)
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"Linear SVM Performance Metrics with C={C_chosen}:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 100, with lowest avg test error: 0.0000
Final Test Error: 0.4824
Linear SVM Performance Metrics with C=100:
Accuracy: 0.5176
Precision: 0.5164
Recall: 0.5176
F1 Score: 0.5170

Confusion Matrix:
[[17 21]
 [20 27]]

Classification Report:
              precision    recall  f1-score   support

           1       0.46      0.45      0.45        38
           2       0.56      0.57      0.57        47

    accuracy                           0.52        85
   macro avg       0.51      0.51      0.51        85
weighted avg       0.52      0.52      0.52        85



### 3. RBF SVM

In [64]:
# Selecting best C for RBF SVM Model (gamma=auto)
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		auto_svm = SVC(kernel='rbf',C=C, gamma='auto')
		auto_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = auto_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = auto_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)

C = 0.01: (avg train error, avg validation error) = (0.7248, 0.7249)
C = 0.1: (avg train error, avg validation error) = (0.7248, 0.7249)
C = 1.0: (avg train error, avg validation error) = (0.3020, 0.3019)
C = 10: (avg train error, avg validation error) = (0.0302, 0.0302)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [68]:
# Fit RBF model on best C (gamma='auto')
lowest_val_error_index = np.argmin(cv_val_avg_score)
# C_chosen = C_values[lowest_val_error_index]
C_chosen = 14.0

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='rbf',C=C_chosen, gamma='auto')
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='auto':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 14.0, with lowest avg test error: 0.0000
Final Test Error: 0.6769
RBF SVM Performance Metrics with C=14.0, gamma='auto':
Accuracy: 0.3231
Precision: 0.3282
Recall: 0.3231
F1 Score: 0.3215

Confusion Matrix:
[[ 9  5  9  2  7  6]
 [ 0 20 10  5  6  6]
 [ 7  5  9  4 10  7]
 [ 2  9  7 14 12  2]
 [ 5  5  4  6 22  6]
 [ 7  4  8  2  8 10]]

Classification Report:
              precision    recall  f1-score   support

           1       0.30      0.24      0.26        38
           2       0.42      0.43      0.42        47
           3       0.19      0.21      0.20        42
           4       0.42      0.30      0.35        46
           5       0.34      0.46      0.39        48
           6       0.27      0.26      0.26        39

    accuracy                           0.32       260
   macro avg       0.32      0.32      0.32       260
weighted avg       0.33      0.32      0.32       260



### 3.1 Sigmoid SVM

In [43]:
# Selecting best C for RBF SVM Model (gamma=auto)
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		auto_svm = SVC(kernel='sigmoid',C=C, gamma='auto', coef0=0.0)
		auto_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = auto_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = auto_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)

C = 0.01: (avg train error, avg validation error) = (0.7852, 0.7852)
C = 0.1: (avg train error, avg validation error) = (0.7852, 0.7852)
C = 1.0: (avg train error, avg validation error) = (0.7315, 0.7315)
C = 10: (avg train error, avg validation error) = (0.6913, 0.6912)
C = 100: (avg train error, avg validation error) = (0.6661, 0.6661)


In [44]:
# Fit Sigmoid model on best C (gamma='auto')
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]
# C_chosen = 100.0

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='sigmoid',C=C_chosen, gamma='auto')
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='auto':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 100, with lowest avg test error: 0.6661
Final Test Error: 0.8308
RBF SVM Performance Metrics with C=100, gamma='auto':
Accuracy: 0.1692
Precision: 0.1987
Recall: 0.1692
F1 Score: 0.1358

Confusion Matrix:
[[ 1 15  0  2  4 16]
 [ 1 18  0  3  3 22]
 [ 2 14  3  2  4 17]
 [ 2 19  4  4  0 17]
 [ 2 23  1  3  2 17]
 [ 0 19  1  0  3 16]]

Classification Report:
              precision    recall  f1-score   support

           1       0.12      0.03      0.04        38
           2       0.17      0.38      0.23        47
           3       0.33      0.07      0.12        42
           4       0.29      0.09      0.13        46
           5       0.12      0.04      0.06        48
           6       0.15      0.41      0.22        39

    accuracy                           0.17       260
   macro avg       0.20      0.17      0.14       260
weighted avg       0.20      0.17      0.14       260



### 3.2 Poly SVM

In [69]:
# Selecting best C for Poly SVM Model (gamma=auto)
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		auto_svm = SVC(kernel='poly',C=C, degree=5, coef0=1)
		auto_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = auto_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = auto_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)

C = 0.01: (avg train error, avg validation error) = (0.6476, 0.6476)
C = 0.1: (avg train error, avg validation error) = (0.0956, 0.0956)
C = 1.0: (avg train error, avg validation error) = (0.0084, 0.0084)
C = 10: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [80]:
# Fit Poly model on best C (gamma='auto')
lowest_val_error_index = np.argmin(cv_val_avg_score)
# C_chosen = C_values[lowest_val_error_index]
C_chosen = 1.0

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='poly',C=C_chosen, degree=5, coef0=1)
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='auto':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 1.0, with lowest avg test error: 0.0000
Final Test Error: 0.6962
RBF SVM Performance Metrics with C=1.0, gamma='auto':
Accuracy: 0.3038
Precision: 0.3109
Recall: 0.3038
F1 Score: 0.3047

Confusion Matrix:
[[ 6  4 13  3  5  7]
 [ 3 19 13  6  0  6]
 [ 7  5 10  5 10  5]
 [ 2  6  7 15 13  3]
 [ 4  5  6  6 21  6]
 [ 8  1  8  5  9  8]]

Classification Report:
              precision    recall  f1-score   support

           1       0.20      0.16      0.18        38
           2       0.47      0.40      0.44        47
           3       0.18      0.24      0.20        42
           4       0.38      0.33      0.35        46
           5       0.36      0.44      0.40        48
           6       0.23      0.21      0.22        39

    accuracy                           0.30       260
   macro avg       0.30      0.29      0.30       260
weighted avg       0.31      0.30      0.30       260



### 4. Custom SVM

In [ ]:
# Define the trigonometric kernel function
def trigonometric_kernel(X, Y, sigma=1.0):
    """
    Trigonometric kernel function as defined in the paper.

    Args:
        X (numpy.ndarray): Matrix of shape (n_samples_X, n_features).
        Y (numpy.ndarray): Matrix of shape (n_samples_Y, n_features).
        sigma (float): Kernel parameter.

    Returns:
        numpy.ndarray: Kernel matrix of shape (n_samples_X, n_samples_Y).
    """
    pairwise_sq_dists = np.sum((X[:, None] - Y[None, :])**2, axis=-1)
    print("Max distance:", np.max(pairwise_sq_dists))
    return np.sin(np.pi / 2 + sigma * pairwise_sq_dists)

# Custom kernel wrapper for scikit-learn
class CustomKernelSVM(SVC):
    def __init__(self, sigma=1.0, **kwargs):
        super().__init__(kernel="precomputed", **kwargs)
        self.sigma = sigma

    def fit(self, X, y):
        # Compute the kernel matrix for training data
        self.X_fit_ = X
        K = trigonometric_kernel(X, X, sigma=self.sigma)
        print("Kernel Matrix (Training):", K)
        print("Kernel Matrix Shape:", K.shape)        
        return super().fit(K, y)

    def predict(self, X):
        # Compute the kernel matrix between training and test data
        K = trigonometric_kernel(X, self.X_fit_, sigma=self.sigma)
        return super().predict(K)



# Initialize and train the SVM with the custom trigonometric kernel
sigma_value = 0.5  # Adjust based on dataset characteristics
svm = CustomKernelSVM(sigma=sigma_value, C=1.0, decision_function_shape="ovr")
svm.fit(X_train, y_train)

# Predict and evaluate
y_pred = svm.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))



## Resize to 64 x 64

In [37]:
print(os.getcwd())
current = os.getcwd()
data_folder = current + "/data"
chess_types_folders = glob.glob(data_folder + "/*")
# print(chess_types_folders)
pieces_info = []
for chess_types in chess_types_folders:
    pieces = glob.glob(f'{chess_types}/*')
    # print(pieces)
    type = chess_types.split("/")[-1]
    for piece in pieces:
        p = {"image_path": piece, "label": type}
        pieces_info.append(p)
chess_df = pd.DataFrame(pieces_info)


/Users/khanhchile/Downloads/School/CSCI 5525/5525Chess


In [38]:
chess_df.head()

,image_path,label
0,/Users/khanhchile/Downloads/School/CSCI 5525/5...,Bishop
1,/Users/khanhchile/Downloads/School/CSCI 5525/5...,Bishop
2,/Users/khanhchile/Downloads/School/CSCI 5525/5...,Bishop
3,/Users/khanhchile/Downloads/School/CSCI 5525/5...,Bishop
4,/Users/khanhchile/Downloads/School/CSCI 5525/5...,Bishop


In [39]:
# Directory to save preprocessed images 64x64
output_dir = "normalized_images_128"
os.makedirs(output_dir, exist_ok=True)

# Normalize and save images
labels = sorted(chess_df['label'].unique())
for label in labels:
    label_dir = os.path.join(output_dir, str(label))  # Create a subdirectory for each label
    os.makedirs(label_dir, exist_ok=True)
    
    for im_pth in chess_df[chess_df['label'] == label]['image_path']:
        # Open and preprocess the image
        image = Image.open(im_pth).convert('L').resize((128, 128))
        image_array = np.array(image) / 255.0  # Normalize to [0, 1]
        print(image_array)
        np.save(os.path.join(label_dir, f"normalized_{os.path.basename(im_pth)}.npy"), image_array)
        # Option

[[0.85098039 0.85490196 0.85490196 ... 0.63529412 0.61568627 0.61176471]
 [0.64313725 0.65098039 0.6627451  ... 0.61960784 0.62352941 0.62745098]
 [0.34509804 0.34901961 0.35294118 ... 0.67058824 0.67058824 0.70980392]
 ...
 [0.28627451 0.28627451 0.29019608 ... 0.39215686 0.40392157 0.41568627]
 [0.29411765 0.29803922 0.30196078 ... 0.39607843 0.4        0.39607843]
 [0.29411765 0.29411765 0.29803922 ... 0.37647059 0.38823529 0.39215686]]
[[0.98431373 0.99607843 1.         ... 1.         0.99607843 0.98431373]
 [0.99215686 0.98431373 1.         ... 1.         0.98431373 0.98823529]
 [1.         0.99215686 0.98431373 ... 0.98431373 0.99215686 1.        ]
 ...
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]]
[[0.18039216 0.17254902 0.34117647 ... 0.82745098 0.81960784 0.82352941]
 [0.17254902 0.18823529 0.43921569 ... 